#Trabajo práctico Deep Learning

En el siguiente trabajo, desarrollaremos una implementación de Question Answering (QA), que buscará responder preguntas referidas al cuento de Jorge Luis Borges, "Emma Zunz" y al texto histórico "El 45", de Félix Luna.

A partir de la utilización de LangChain, un framework que permite interactuar con diversos modelos de lenguajes (LLMs), exploraremos la implementación de una secuencia de QA con dos de los posibles métodos existentes: en primera instancia, utilizando VectorstoreIndexCreator; en segunda instancia, utilizando load_qa_chain y ConversationalRetrievalChain.

#VectorstoreIndexCreator

En su libro "*¿Hola? Un réquiem para el teléfono*", Martín Kohan realiza un interesante análisis de la obra de Borges, que transcribimos a continuación (sintetizado), ya que permitirá dar contexto a las preguntas que intentaremos responder con nuestro modelo.

"Emma Zunz es un cuento del cuerpo y las experiencias. (...) Emma Zunz primero narra, primero planifica, pero después va a salir a vivir todo eso que primero narró: va a traspasar esa previa narración al mundo de las experiencias. Y eso implica ni más ni menos que poner el cuerpo en juego.

¿Qué cuerpo? (...) El cuerpo que reaccionó a la lectura de la carta que le hacía saber a Emma que su padre había muerto en Brasil.

(...) Es ese cuerpo, ni más ni menos, (...) el que Emma Zunz va a poner en juego para afrontar dos experiencias viscerales: un acto sexual (no cualquier acto sexual, sino el que ella, virgen y temerosa casi hasta lo patológico, va a concretar con un marinero cualquiera fingiéndose una prostituta) y un acto criminal (va a matar a Aaron Loewenthal, el dueño de la fábrica donde ella trabaja, el verdadero responsable del desfalco por el que se acusó a su padre y se lo hizo caer en desgracia). (...) el cuerpo se va a poner en juego dos veces, una para el ultraje y otra para el asesinato en procura de una coartada que en principio servirá para resolver la historia: el cuerpo será la prueba material de un abuso sexual que Emma endilgará a Aaron Loewenthal para justificar así el haberlo matado.

(...) Así, si en el comienzo el cuerpo de Emma llevaba una narración (un plan) a la realidad de los hechos, en el final será lo que imprimirá verdad a una narración (una coartada) de hechos que en realidad no ocurrieron (o no de la forma en que habrá de contarlos Emma, falseando "las circunstancias, la hora y uno o dos nombres propios").

(...) Para lograr el encuentro con el marinero, Emma se hace pasar por prostituta. (...) Para lograr el encuentro con Loewenthal, Emma (...) tiene que hacerle creer que va a delatar a quienes están detrás de la huelga de la que se rumorea en la fábrica; lo logra, él le cree, la recibe en su escritorio. Pero si la primera de estas dos tretas del hacer creer se resuelve como astucia del cuerpo, (...) en la segunda aparece un elemento distinto, que llegará a ser decisivo en el desenlace del cuento. ¿Qué aparece? Un teléfono. A Loewenthal, Emma lo llama por teléfono. (...) Si el marinero en el puerto habrá de creerle al cuerpo, Loewenthal, en el teléfono, habrá de creerle a la voz. Pero no a cualquier voz, sino a la instrumentación de una voz sin cuerpo, de una voz presente en el cuerpo ausente, que es lo propio del teléfono. "Emma Zunz" es un cuento sobre la verdad y la verosimilitud, que se resuelve en una verdad del cuerpo. Pero con el teléfono se introduce algo distinto, que es y no es ya el cuerpo: se introduce la verdad de la voz. A Emma le tiembla la voz de verdad cuando hace el llamado a Loewenthal. Una verdad puesta en desvío: no le tiembla porque va a delatar, le tiembla porque va a matarlo. También la verdad del cuerpo, después de todo, se verá a su vez desviada: el ultraje vivido con el marinero en el puerto irá a parar a Loewenthal en su escritorio.

El teléfono es decisivo en el cuento, porque aparece también en el final. Y, en el final, decide la historia.

(...) El teléfono con el que antes Loewenthal atendió su llamado y que ahora emplea para llamar a la policía y contar lo que pasó: "Desordenó el diván, desabrochó el saco del cadáver, le quitó los quevedos salpicados y los dejó sobre el fichero. Luego tomó el teléfono y repitió lo que tantas veces repitiría, con esas y con otras palabras: Ha ocurrido una cosa que es increíble... El señor Loewenthal me hizo venir con el pretexto de la huelga... Abusó de mi, lo maté...".

Borges especifica que Emma va a repetir su historia. (...) Y deja ver que es de este modo como va a imponerse a todos: va a imponerse *como narración.* Emma ofrece su cuento y le creen. Le creen, y puede inferirse por eso que prescinden de examinar el cuerpo, que es lo que ella había previsto. Si al comienzo hubo una narración (un plan) que requirió poner el cuerpo en juego, en el final hay otra narración (una denuncia) que se impondrá como tal y, al imponerse, eximirá al cuerpo de tener que ponerse en juego otra vez (como prueba del abuso denunciado).

Es otra clase de verdad la que termina por imponerse. (...) Más que la verdad objetivada en el cuerpo (...), lo que se impone es una verdad subjetiva (la del pudor, la del odio) que encuentra su tan plena eficacia como verdad de la voz: "Verdadero era el tono de Emma Zunz".

Esa verdad, la de la voz, se impone hasta tal punto que llega incluso a relegar la del cuerpo. Verdad de la voz, ya sin el cuerpo: no es casual que el relato de los hechos los haga Emma Zunz por teléfono. El teléfono es en verdad su gran aliado. El que permite que su demorada venganza se concrete por fin, con éxito. Y el que permite que el ardid de su coartada se imponga y que su crimen, ya que lo fue, quede perfectamente cubierto, perfectamente impune".

En primera instancia, importamos las librerías y frameworks necesarios para implementar nuestro ejercicio.

In [ ]:
!pip install langchain
!pip install pypdf
!pip install faiss-cpu
!pip install openai
!pip install tiktoken
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.5/254.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 77.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.1/965.1 kB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 7.8

Luego, incorporamos la API Key que nos permitirá darle usabilidad al mismo.

In [ ]:
import os
from getpass import getpass
OPENAI_API_KEY = getpass("OPENAI_API_KEY")
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

OPENAI_API_KEY··········


Cargaremos nuestro documento: el cuento de Borges, "Emma Zunz".

In [ ]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader('/content/Zunz.pdf')
pages = loader.load_and_split()

In [ ]:
pages[0]

Document(page_content='Emma Zunz  \nJorge Luis Borges  \n  \nEl catorce de enero de 1922, Emma Zunz, al volver d e la fábrica de tejidos Tarbuch \ny Loewenthal, halló en el fondo del zaguán una cart a, fechada en el Brasil, por la \nque supo que su padre había muerto. La engañaron, a  primera vista, el sello y el \nsobre; luego, la inquietó la letra desconocida. Nue ve diez líneas borroneadas \nquerían colmar la hoja; Emma leyó que el señor Maie r había ingerido por error una \nfuerte dosis de veronal y había fallecido el tres d el corriente en el hospital de Bagé. \nUn compañero de pensión de su padre firmaba la noti cia, un tal Feino Fain, de Río \nGrande, que no podía saber que se dirigía a la hija  del muerto. \nEmma dejó caer el papel. Su primera impresión fue d e malestar en el vientre y en \nlas rodillas; luego de ciega culpa, de irrealidad, de frío, de temor; luego, quiso ya \nestar en el día siguiente. Acto continuo comprendió  que esa voluntad era inútil \nporque la muerte de

In [ ]:
print(f'Tenes {len(pages)} páginas en tu documento y {len(pages[0].page_content)} caracteres.')

Tenes 4 páginas en tu documento y 2382 caracteres.


Generamos el índice sobre la data, utilizando el mejor y más popular que existe en la actualidad, que es el VectorStore. Teniendo en cuenta que no se trata de un set de datos extenso, no realizaremos un split de los datos en varios documentos más pequeños para simplificar y hacer más eficiente la búsqueda de OpenAI ante cada una de nuestras preguntas.

In [ ]:
from langchain.indexes import VectorstoreIndexCreator
index = VectorstoreIndexCreator().from_loaders([loader])

Realizamos una serie de preguntas.

In [ ]:
query = "¿Qué fue lo primero que vio Emma Zunz al volver de la fabrica de tejidos?"
index.query(query)

' Emma Zunz vio una carta en el fondo del zaguán al volver de la fábrica de tejidos.'

Como puede verse, el índice pudo responder nuestra pregunta (un tanto descriptiva y sencilla) de forma correcta, ya que la respuesta surge de las primeras líneas del texto. Veamos qué ocurre cuando sugerimos realizar cierta combinación e inteligencia sobre los datos del texto.

In [ ]:
query2 = "¿Quién era Emma Zunz?"
index.query(query2)

' Emma Zunz era la hija de Manuel Maier, quien había muerto en el hospital de Bagé tras ingerir una fuerte dosis de veronal por error. Emma descubrió la noticia de la muerte de su padre al encontrar una carta en el zaguán de la fábrica de tejidos Tarbuch y Loewenthal, donde trabajaba.'

¡Aquí también el índice resolvió correctamente! Emma Zunz era la hija de Manuel Maier. Inclusive, en la respuesta se incorpora el contexto sobre la muerte, indicando cómo se enteró de la noticia fatal.

Veamos también qué ocurre realizando otras preguntas.

In [ ]:
query3 = "¿Quién mató a Aaron Loewenthal?"
index.query(query3)

' Emma Zunz mató a Aaron Loewenthal.'

In [ ]:
query4 = "¿Quién fue el acusado del asesinato de Aaron Loewenthal?"
index.query(query4)

' Emma Zunz.'

In [ ]:
query5 = "¿Quién cree la policía que fue el asesino de Aaron Loewenthal?"
index.query(query5)

' La policía cree que Emma Zunz fue el asesino de Aaron Loewenthal.'

In [ ]:
query52 = "¿Cuál fue la coartada usada por Emma Zunz para justificar su asesinato?"
index.query(query52)

' Emma Zunz usó la coartada de que había sido abusada por el señor Loewenthal y que lo mató para vengar el ultraje padecido por su padre.'

En los cuatro casos el índice responde de forma correcta. Pese a que se trató de una historia inventada por Emma Zunz, se supo reconocer quién fue el asesino real y también cuál fue el pretexto que utilizó la asesina para justificar la muerte de Loewenthal.

In [ ]:
query6 = "¿Qué medio fue utilizado para denunciar la muerte de Aaron Loewenthal?"
index.query(query6)

' Emma Zunz usó el teléfono para denunciar la muerte de Aaron Loewenthal.'

Si preguntamos cómo comunicó a la policía el fallecimiento de Loewenthal, el índice también contesta de forma correcta.

In [ ]:
query7 = "¿Cuál era la nacionalidad del marinero?"
index.query(query7)

' No se sabe.'

El índice volvió a responder correctamente. En el texto se menciona lo siguiente: "El hombre, sueco o finlandés, no hablaba español". Como no puede saberse si era sueco o finlandés, respondió que no se sabe.

In [ ]:
query8 = "¿Cómo logra Emma llegar al despacho de Loewenthal?"
index.query(query8)

' Emma llama por teléfono a Loewenthal, insinuando que desea comunicar algo sobre la huelga, y promete pasar por el escritorio al oscurecer.'

In [ ]:
query9 = "¿Cuál es el plan de Emma para matar a Loewenthal?"
index.query(query9)

' Emma planea forzar a Loewenthal a confesar su culpa y luego matarlo con un solo balazo en el pecho.'

In [ ]:
query10 = "¿Por qué Emma fingió ser abusada por Loewenthal?"
index.query(query10)

' Emma fingió ser abusada por Loewenthal para justificar su acción de matarlo y evitar ser castigada.'

In [ ]:
query11 = "¿Qué hizo Emma el día previo al asesinato?"
index.query(query11)

' Emma trabajó hasta las doce, se acostó después de almorzar, recapituló su plan, leyó una carta de Fain, se arrepintió de un acto de soberbia, se vistió, tomó un Lacroze al oeste, viajó por barrios decrecientes y opacos, y se apeó en una de las bocacalles de Warnes.'

In [ ]:
query12 = "¿Qué dijo Emma al llamar a la policía?"
index.query(query12)

' Emma dijo que había ocurrido algo increíble, que el señor Loewenthal la había hecho venir con el pretexto de la huelga y que él la había abusado, por lo que ella lo había matado.'

Las respuestas generadas de la query 8 a la 12, fueron correctas (al igual que todas las anteriores que utilizamos para probar su funcionamiento).

In [ ]:
query13 = "¿Dónde vivía Emma?"
index.query(query13)

' Emma vivía en la fábrica de tejidos Tarbuch y Loewenthal.'

Si bien no es lo usual, el índice también responde algunas preguntas simples de forma errónea. En el texto se especifica que "Emma vivía por Almagro, en la calle Liniers". Aquí fueron confundidas las entidades y se afirmó que Emma vivía en la fábrica de tejidos.

In [ ]:
query14 = "¿Dónde vivía Emma? Una pista: no es en la fábrica de tejidos."
index.query(query14)

' Emma vivía en una de las bocacalles de Warnes.'

In [ ]:
query15 = "¿Emma vivía por Almagro?"
index.query(query15)

' Sí, Emma vivía por Almagro.'

In [ ]:
query16 = "¿En qué calle de Almagro vivía Emma?"
index.query(query16)

' Emma vivía por Almagro, en la calle Liniers.'

Al brindarle un poco más de contexto (quizás algo exagerado), el índice respondió de forma correcta nuestras preguntas. Es destacable, también, que tuvo una segunda oportunidad de responder de forma correcta y falló nuevamente, respondiendo "en una de las bocacalles de Warnes".

# load_qa_chain y ConversationalRetrievalChain


¿Qué ocurriría si en lugar de usar el índice VectorStore utilizáramos algún otro método para resolver este problema? VectorStore es una interfaz que permite interactuar a más alto nivel, pero existen otros mecanismos de utilización, como load_qa_chain que permiten particionar o *splittear* el texto tantas veces como sea necesario. Es un mecanismo que aplica eficiencia computacional y permite realizar estos procedimientos incluso con varios textos a la vez.

Comenzaremos por importar el texto.

In [ ]:
from langchain.chains.question_answering import load_qa_chain

loader2 = PyPDFLoader('/content/El 45 [Felix Luna].pdf')
document = loader2.load()

Dado que se trata de un texto mucho más extenso que el primero (el de Borges), lo que haremos aquí es utilizar el tipo de cadena *map_reduce*. A partir de ello, el texto será separado en batches, y la query será pasada a cada batch por separado. La respuesta final será la que considere más aceptable entre todas las que obtenga.

In [ ]:
from langchain.llms import OpenAI

In [ ]:
chain = load_qa_chain(llm=OpenAI(), chain_type='map_reduce')
query17 = '¿Cuál era el último elemento que operaba en contra de Perón dentro del Ejército entre abril y septiembre de 1945?'
chain.run(input_documents = document, question=query17)

' El último elemento que operaba en contra de Perón dentro del Ejército entre abril y septiembre de 1945 fue el general Edelmiro Farrell.'

Esto es lo que escribió Luna en el libro: "Y un elemento más incidía en la creación de la resistencia que se estaba articulando lentamente en el Ejército: su relación amorosa con Eva Duarte. Desde los primeros meses de 1944 esta *liaison* era notoria y, para muchos de sus camaradas, inaceptable. Que este coronel viudo recreara su cuarentena con una actriz, era irreprochable e incluso estaba dentro de las más prestigiantes tradiciones del oficio.
—Me reprochan que ande con una actriz... ¿Y qué quieren, que ande con un actor? —solía bromear Perón con grueso humorismo cuartelero".

Como puede verse, la respuesta obtenida no fue la que esperábamos. Utilizaremos otro tipo de cadena (refine), el cual esperamos pueda refinar, valga la redundancia, la respuesta obtenida por medio de un análisis de los batches mucho más exhaustivo.

In [ ]:
chain = load_qa_chain(llm=OpenAI(), chain_type='refine')
query17 = '¿Cuál era el último elemento que operaba en contra de Perón dentro del Ejército entre abril y septiembre de 1945?'
chain.run(input_documents = document, question=query17)

'\n\nEl último elemento que operaba en contra de Perón dentro del Ejército entre abril y septiembre de 1945 fue el general de brigada Tomás Ramón Jofré. Él lideró la Unión Democrática, una coalición de sectores opositores compuesta por varios partidos políticos, organizaciones democráticas, manifestantes en la calle, estudiantes de la FUA, sindicalistas, empresarios, estancieros, opositores ansiosos por construir un régimen democrático, figuras apolíticas, políticos oscuros desplazados del escenario nacional, federaciones universitarias, centros de estudiantes, organizaciones sindicales que se desafiliaron de la CGT, grupo de periodistas, entre ellos Marcial Rocha Demaría y Eduardo J. Pacheco, empresariales de ganaderos, industriales, comerciantes, diput'

Quizás la pregunta no está siendo lo suficientemente precisa para poder ser reconocida por nuestro modelo. La respuesta que esperamos, como transcribimos más arriba, es su relación con Eva Perón. Intentaremos reprocesar la pregunta, con el primero de los métodos, intentando precisarla aún más.

In [ ]:
chain = load_qa_chain(llm=OpenAI(), chain_type='map_reduce')
query18 = 'La relación amorosa o liaison de Juan Domingo Perón con Eva Perón, ¿generó resistencia dentro del Ejército en los primeros meses de 1944?'
chain.run(input_documents = document, question=query18)

' No, la relación amorosa o liaison de Juan Domingo Perón con Eva Perón no generó resistencia dentro del Ejército en los primeros meses de 1944.'

Incluso siendo mucho más precisos en el contenido de la pregunta, la respuesta no fue la que esperábamos. Intentaremos realizar la pregunta con el último de los métodos que conocemos: *ConversationalRetrievalChain*.

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 0)
text = text_splitter.split_documents(document)
embeddings = OpenAIEmbeddings()
db = Chroma.from_documents(text, embeddings)
retriever = db.as_retriever(search_type = "similarity", search_kwargs={"k":2})
qa = ConversationalRetrievalChain.from_llm(OpenAI(), retriever)

In [ ]:
chat_history = []
query19 = "La relación amorosa o liaison de Juan Domingo Perón con Eva Perón, ¿generó resistencia dentro del Ejército en los primeros meses de 1944?"
result = qa({"question": query19, "chat_history": chat_history})

In [ ]:
result["answer"]

' Sí, generó resistencia dentro del Ejército en los primeros meses de 1944.'

In [ ]:
chat_history = [(query19, result["answer"])]
query19 = "¿Por qué generó resistencia?"
result = qa({"question": query19, "chat_history": chat_history})

In [ ]:
chat_history

[('¿Por qué generó resistencia dentro del Ejército en los primeros meses de 1944?',
  ' Las razones para la resistencia dentro del Ejército en los primeros meses de 1944 respecto a la relación amorosa de Juan Domingo Perón con Eva Perón eran que era inaceptable para muchos de sus camaradas, considerándolo como una falta de ética militar, y además su relación con ella era muy pública.')]

In [ ]:
result["answer"]

' La resistencia dentro del Ejército se debía a que su relación amorosa con Eva Duarte era inaceptable para muchos de sus camaradas, ya que era atentar contra la ética militar exhibirse públicamente con ella, presentarla a sus amigos, vivir en su casa y hacerla participar en sus tertulias políticas.'

Finalmente, hemos conseguido la respuesta que esperábamos. El método de *ConversationalRetrievalChain*, que además adquiere la particularidad de guardar la historia, nos permitió generar una respuesta acertada, acorde a lo planteado por Luna en su texto.

# Conclusión

En resumen, podemos afirmar que esta implementación logra realizar un más que aceptable recocimiento de entidades a lo largo del texto, respondiendo en la gran mayoría de los casos de la forma en que se espera.

Sin embargo, es destacable que no es una herramienta que pueda funcionar con total independencia: está visto que aún maneja algunas imprecisiones que pueden resultar muy peligrosas si se otorga una confianza ciega a las respuestas obtenidas.